In [19]:
import random
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import sys
import numpy

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\ruribeg\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


Hites

In [20]:
url = "https://www.hites.com/tecnologia/computacion/notebook/?start=0&sz=70"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,"html.parser")
bloque = soup.find_all("div",{"class":"col-6 col-lg-4 px-0"})

In [21]:
nombres = list()
precioaux = list()
precio = list()
codigos = list()
encontrarindice = list()
caracteristica = list()
almacenamientoaux = list()
almacenamiento = list()
procesadoraux = list()
procesador = list()
ramaux = list()
ram = list()
precios = list()

for i in bloque:
    nombre = i.find("a",{"class":"link product-name"})
    caract = i.find_all("li",{"class":"attribute-values"})
    precios = i.find("span",{"class":"value"})
    codigo = i.find("span",{"class":"d-block product-sku"})
    nombres.append(nombre.text)
    caracteristica.append(caract)  
    precioaux.append(precios.text)
    codigos.append(codigo.text)

""""Obtener cada elemento de almacenamiento, memoria ram y procesador definiendo un valor auxiliar 
    para almacenamiento en caso de no encontrarse en la pagina"""
for i in range(len(caracteristica)):
    if (len(caracteristica[i])==2):
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][0])
        procesadoraux.append(caracteristica[i][1])
        
    else:
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][1])
        procesadoraux.append(caracteristica[i][2])

#Insertar elementos de procesador y memoria ram en las listas como texto, en el caso de almacenamiento como str
for i in range(len(almacenamientoaux)):
    if(type(almacenamientoaux[i]==str)):
        almacenamiento.append(almacenamientoaux[i])
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)
        
    else:
        almacenamiento.append(almacenamientoaux[i].text)
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)  

#Modificar los elementos de almacenamiento a formato text
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].text
    

#Modicar valor auxiliar de almacenamiento en caso de que no se encontrará en la pagina          
for i in range(len(almacenamiento)):
    if ('RAM' in almacenamiento[i])==True:
        almacenamiento[i]="Almacenamiento no especificado"
        
#Reemplazar los saltos de linea del formato text a expacios en blanco para obtener solo los datos deseados
for i in range(len(caracteristica)):
    almacenamiento[i] = almacenamiento[i].replace('\n','')
    ram[i] = ram[i].replace('\n','')
    procesador[i] = procesador[i].replace('\n','')

#Encontrar el indice del primer slash para poder separar el nombre original sin los componentes
for i in range(len(nombres)):
    encontrarindice.append(nombres[i].find("/"))

#Dejar solo el nombre de los notebooks dentro de la lista
for i in range(len(nombres)):
    nombres[i] = nombres[i][0:encontrarindice[i]-1]
    
#Eliminación de caracteres especiales
for i in range(len(precioaux)):
    precioaux[i] = precioaux[i].replace("Price reduced from", "")
    precioaux[i] = precioaux[i].replace("\n","")
    precioaux[i] = precioaux[i].replace("to","")
    precioaux[i] = precioaux[i].replace("$","")
    precioaux[i] = precioaux[i].replace(".","")
    precioaux[i] = precioaux[i].replace("Normal","")
    precioaux[i] = precioaux[i].replace("Oferta","")
    precio.append(precioaux[i])
    
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    
for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("Procesador:", "")
    
for i in range(len(ram)):
    ram[i] = ram[i].replace("Memoria RAM:", "")

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("Código: ", "")
    codigos[i] = codigos[i].replace("\n","")  

In [22]:
df = pd.read_csv('Notebooks.csv')
x = datetime.today().strftime('%Y-%m-%d')

for i in range(len(caracteristica)):
    df = df.append({'Nombre': nombres[i], 'Codigo':codigos[i], 'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precio[i], 'Fecha':x},ignore_index=True)

PC FACTORY

In [23]:
url_base=[]
url_notebooks=[]
url1 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636"
url2 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636&pagina=2"

url_base.append(url1)
url_base.append(url2)


for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="product"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


In [24]:
marca = []
nombres = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigos = []
ram = []
precios = []

for i in url_notebooks:
    maux = 0
    moaux = 0
    almaux = 0
    proaux = 0
    ramaux = 0
    driver.get(i)
    bloque_codigos = driver.find_elements_by_xpath('//p[@class="link color-dark"]')

    codigo = bloque_codigos[1]
    codigos.append(codigo.text)

    precio = driver.find_element_by_xpath('//div[@class="price-xl color-primary-1"]').text
    precios.append(precio)

    
    
    caracteristicas = driver.find_elements_by_xpath("//li")

    for i in range(len(caracteristicas)):
        caracteristicas[i] = caracteristicas[i].text

    for i in caracteristicas:
        if(i.find("Marca")!=-1):
            maux = maux + 1
            nombre_marca = i
        if(i.find("Modelo")!=-1):
            moaux = moaux + 1
            nombre_modelo = i
        if(i.find("Almacenamiento")!=-1):
            almaux = almaux + 1
            nombre_almacenamiento = i
        if(i.find("RAM:")!=-1 or i.find("Memoria:")!=-1 or i.find("Memoria Total:")!=-1):
            ramaux = ramaux + 1
            nombre_ram = i
        if(i.find("Procesador:")!=-1):
            proaux = proaux + 1
            nombre_procesador = i

    if(maux >= 1):
        marca.append(nombre_marca)
    elif(maux == 0):
        marca.append("Marca no encontrada")
    if(moaux >= 1):
        modelo.append(nombre_modelo)
    elif(moaux == 0):
        modelo.append("Modelo no encontrado")
    if(almaux >= 1):
        almacenamiento.append(nombre_almacenamiento)
    elif (almaux == 0 ):
        almacenamiento.append("Almacenamiento no encontrado")
    if(ramaux >= 1):
        ram.append(nombre_ram)
    elif(ramaux == 0):
        ram.append("Memoria ram no encontrada")
    if(proaux >= 1):
        procesador.append(nombre_procesador)
    elif(proaux == 0):
        procesador.append("Procesador no encontrado")

    sleep(random.uniform(2,4))



for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("\xa0", " ")
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    almacenamiento[i] = almacenamiento[i].lstrip()

for i in range(len(marca)):
    marca[i] = marca[i].replace("\xa0"," ")
    marca[i] = marca[i].replace("Marca:","")
    marca[i] = marca[i].lstrip()

for i in range(len(modelo)):
    modelo[i] = modelo[i].replace("\xa0", " ")
    modelo[i] = modelo[i].replace("Modelo:", "")
    modelo[i] = modelo[i].lstrip()

for i in range(len(ram)):
    ram[i] = ram[i].replace("\xa0", " ")
    ram[i] = ram[i].replace("Memoria RAM:", "")
    ram[i] = ram[i].replace("Memoria:", "")
    ram[i] = ram[i].replace("Memoria Total:", "")
    ram[i] = ram[i].lstrip()

for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("\xa0"," ")
    procesador[i] = procesador[i].replace("Procesador:","")
    procesador[i] = procesador[i].lstrip()

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ","")
    precios[i] = precios[i].replace("$","")
    precios[i] = precios[i].replace(".","")

for i in range(len(marca)):
    nombres.append(marca[i] + " " + modelo[i])

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("ID ","")

x = datetime.today().strftime('%Y-%m-%d')

for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombres[i],'Codigo':codigos[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], 'Fecha':x},ignore_index=True)

Falabella

In [25]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,3):
    url = "https://www.falabella.com/falabella-cl/category/cat2028/Notebooks-Gamers?page="+str(i)
    url_base.append(url)


##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="jsx-4001457643 search-results-list"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []
for i in url_notebooks:
    nombre = []
    valor = []
    driver.get(i)
    sleep(2)
    boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
    sleep(random.uniform(4,5))
    nombre_aux = driver.find_element_by_xpath('//div[@class="jsx-3686231685 product-name fa--product-name"]').text
    elementos = driver.find_elements_by_xpath('//tr[@class="jsx-428502957"]')
    try:
        precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-340449923 normal    "]').text
        
    except:
        try:
            precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-340449923 normal    "]').text
        except:
            try:
                precio = driver.find_element_by_xpath('//h2[@class="jsx-2034229452 title6 mkp-secondary    "]').text
            except:
                precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-2612542277 normal     "]').text
    codigos = driver.find_element_by_xpath('//span[@class="jsx-3408573263"]').text
    for j in elementos:
        
        nombres = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-name"]').text
        valores = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-value"]').text
        nombre.append(nombres)
        valor.append(valores)
    
    if("Procesador" in nombre)==True:
        index_procesador = nombre.index("Procesador")
        procesador.append(valor[index_procesador].upper())
        
    if("Procesador" in nombre)==False:
        procesador.append("Procesador no encontrado")
        
    if("Unidad de estado sólido SSD" in nombre)==True:
        index_almacenamiento = nombre.index("Unidad de estado sólido SSD")
        almacenamiento.append(valor[index_almacenamiento].upper())
        
    if("Unidad de estado sólido SSD" in nombre)==False:
        almacenamiento.append("Almacenamiento no encontrado")
        
    if("Memoria RAM" in nombre)==True:
        index_ram= nombre.index("Memoria RAM")
        ram.append(valor[index_ram].upper())
        
    if("Memoria RAM" in nombre)==False:
        ram.append("Memoria Ram no encontrada") 
        
    
    precios.append(precio)
    codigo.append(codigos)
    nombre_equipo.append(nombre_aux)
        

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

for i in range(len(codigo)):
    codigo[i] = codigo[i].replace("Código del producto: ","")


x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i],'Codigo':codigo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

HP

In [26]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,6):
    url = "https://www.hp.com/cl-es/shop/notebooks/notebooks-intel.html?gclsrc=aw.ds&p="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//li[@class="item product product-item"]')
    for i in notebook:
        try:
            link = i.find_element_by_tag_name('a').get_attribute('href')
        except:
            x = 2
        url_notebooks.append(link)

##Listas que almacenaran datos encontrados dentro del sitio web
marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []

for i in url_notebooks:
    nombres = []
    valores = []
    sleep(2)
    driver.get(i)
    try:
        boton = driver.find_element_by_xpath('//div[@class="data item title" and @id="tab-label-additional"]').click()
    except:
        try:

            boton = driver.find_element_by_xpath('//a[@class="data switch" and @data-toggle="trigger"').click()
        except:
            x = "boton no encontrado"
    sleep(5)
    try:
        name = driver.find_element_by_xpath('//span[@class="base" and @data-ui-id="page-title-wrapper"]').text
    except:
        name = "Nombre no encontrado"
    try:
        price = driver.find_element_by_xpath('//span[@class="price"]').text
    except:
        price = "Precio no encontrado"
    
    try:
        code = driver.find_element_by_xpath('//div[@class="value " and @itemprop="sku"]').text
    except:
        code = "Codigo no encontrado"
    bloque = driver.find_elements_by_xpath('.//tr[@class="item"]')
    for j in bloque:
        nombre = j.find_element_by_xpath('.//th[@class="col label"]').text
        valor = j.find_element_by_xpath('.//td[@class="col data"]').text
        nombres.append(nombre)
        valores.append(valor)
        
    if ("Procesador" in nombres)==True:
        index_procesador = nombres.index("Procesador")
        procesador.append(valores[index_procesador])
        
    elif ("Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")
        
    if ("Memoria, estándar" in nombres)==True:
        index_ram = nombres.index("Memoria, estándar")
        ram.append(valores[index_ram])
        
    elif ("Memoria, estándar" in nombres)==False:
        ram.append("Memoria ram no especificada")
        
    if("Descripción del disco duro" in nombres)==True:
        index_almacenamiento = nombres.index("Descripción del disco duro")
        almacenamiento.append(valores[index_almacenamiento])
    
    elif("Descripción del disco duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    nombre_equipo.append(name)
    precios.append(price)
    codigo.append(code)

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")
    if(precios[i]==""):
        precios[i] = "Precio no especificado"

In [27]:
#Llenado de cada uno de los campos
x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigo[i], 'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precios[i], "Fecha":x},ignore_index=True)

In [28]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,8):
    url = "https://simple.ripley.cl/tecno/computacion/notebooks?page="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

##Obtención de datos dentro de cada una de las páginas del sitio web, almacenados en listas auxiliares

nombre_equipo = []
precios = []
almacenamiento = []
procesador = []
ram = []
codigos = []

for i in url_notebooks:
    nombres = []
    valores = []        
    driver.get(i)
    sleep(5)
    try:
        boton = driver.find_element_by_xpath('//div[@aria-controls="panel-Especificaciones"]').click()
    except:
        boton = "holi"
    sleep(2)
    try:
        ver = driver.find_element_by_xpath('//button[@class="btn-loading  rpl-show-more btn-outline-grey "]').click()
    except:
        x=2
    bloque = driver.find_elements_by_xpath('//tr')
    bloque_nombre = driver.find_elements_by_xpath('//section[@class="product-header hidden-xs"]')
    try:
        precio = driver.find_element_by_xpath('//dt[@class="product-price"]')
    except:
        try:
            precio = driver.find_element_by_xpath('//dt[@class="product-price product-price__line-thru"]')
        except:
            precio = "Precio no encontrado"
    for i in bloque_nombre:
        nombre = i.find_element_by_xpath('.//h1')
        codigo = i.find_element_by_xpath('.//span[@class="sku sku-value"]')
    for i in bloque:
        elementos = i.find_elements_by_xpath('.//td')
        for j in range(len(elementos)):
            if (j==0):
                nombres.append(elementos[j].text)
            if (j==1):
                valores.append(elementos[j].text)
                
    if ("Tipo Procesador" in nombres)==True:
        index_procesador = nombres.index("Tipo Procesador")
        procesador.append(valores[index_procesador])

    if ("Tipo Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")

    if ("Memoria Ram" in nombres)==True:
        index_ram = nombres.index("Memoria Ram")
        
        ram.append(valores[index_ram])
        
    if ("Memoria Ram" in nombres)==False:
        ram.append("Memoria Ram no especificada")
        
    if ("Capacidad Disco Duro" in nombres)==True:
        idex_almacenamiento = nombres.index("Capacidad Disco Duro")
        almacenamiento.append(valores[idex_almacenamiento])
        
    if ("Capacidad Disco Duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    codigos.append(codigo.text)
    try:
        precios.append(precio.text)

    except:
        precios.append(precio)
    nombre_equipo.append(nombre.text)


for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigos[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

In [29]:
df = df.drop(['Unnamed: 0'],axis=1)
df.to_csv('Notebooks.csv')


In [30]:
df

,Nombre,Codigo,Almacenamiento,Procesador,Memoria Ram,Precio,Fecha
0,Notebook Asus X515ma-br576t,861599001,500 Gb,Intel Celeron,4 Gb,309990,2021-09-15
1,Notebook Reacondicionado Hp 14-dq1043c I3,851066001,256 Gb,Intel Core I3,8 Gb,399990,2021-09-15
2,Notebook Gamer Asus Tuf Gaming Fx506lh-hn082t,861708001,512 Gb,Intel Core I5,8 Gb,769990,2021-09-15
3,Notebook Lenovo Ideapad 3,841656001,1 Tb,Amd Athlon,8 Gb,399990,2021-09-15
4,Notebook Asus X415ea-eb742t,861710001,512 Gb,Intel Core I7,8 Gb,769990,2021-09-15
...,...,...,...,...,...,...,...
6521,NOTEBOOK IDEAPAD D330 INTEL 4GB RAM 64GB 10'' ...,MPM00031467106,Almacenamiento no especificado,Procesador no especificado,Memoria Ram no especificada,369990,2021-09-28
6522,NOTEBOOK HP PAVILION X360 14-DY0002LA PENTIUM ...,MPM00031393952,256 GB SSD,Intel Pentium,4GB,429990,2021-09-28
6523,NOTEBOOK THINKPAD L13 / INTEL CORE I7 10° GEN ...,MPM00031394044,512 GB SSD,Intel Core i7,16GB,969990,2021-09-28
6524,NOTEBOOK GAMING OMEN 15-EK0007LA CORE I7 16GB ...,MPM00031394146,512 GB SSD,Intel Core i7,16GB,1649990,2021-09-28


array(['500 Gb', '256 Gb', '512 Gb', '1 Tb', '128 Gb', ' 512 Gb',
       '1 Tb + 128 Gb', '64 Gb', '512 GB', '32GB eMMC',
       'SATA de 500 GB, 5400 rpm', '500GB SATA 5400RPM 2.5" HDD',
       '1TB Disco duro SATA de 5400 RPM', 'HDD 1 TB 5400RPM',
       '256GB SSD / M.2 NVMe™ PCIe® 3.0', 'SATA de 1 TB y 5400 rpm',
       '256GB SSD PCI Express',
       'Unidad de estado sólido Intel® PCIe® NVMe™ M.2 de 256 GB',
       'SSD PCIe® NVMe™ M.2 de 256GB', '256GB M.2 NVMe™ PCIe® 3.0 SSD',
       '256GB SSD PCIe® NVMe™ M.2', 'SSD N4VMe PCIe de 256GB',
       'SSD NVMe PCIe de 256GB',
       'Unidad de estado sólido PCIe® NVMe™ M.2 de 256 GB',
       'SSD NVMe PCIe de 512 GB', '256GB SSD PCIe®  NVMe™ M.2',
       'Unidad de estado sólido M.2 PCIe® NVMe™ de 512 GB',
       '512GB M.2 NVMe™ PCIe® 3.0 SSD',
       '256GB SSD / Unidad de estado sólido Intel',
       '512GB SSD M.2 2280 PCIe 3.0x4 NVMe', '512GB SSD PCI Express',
       'Unidad de estado sólido PCIe® NVMe™ M.2 de 256 GB ',
       

101

[]

In [34]:
data

'\n<!DOCTYPE html>\n<html lang="es">\n<head>\n<!--[if gt IE 9]><!-->\n<script type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/js/imageReplacement.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/js/main.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/js/menuV2.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/lib/js/picturefill.min.js"></script>\n<script type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/lib/js/jRespond.min.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/js/search.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-HITES-Site/-/default/v1632834995966/js/recommendedProdu

In [35]:
data2

''